In [1]:
import numpy as np
import fnmatch, os, sys
sys.path.append('D:\\OneDrive\\codes\\xds')

### Loading data from an xds format file

In [2]:
from xds import lab_data, list_to_nparray, smooth_binned_spikes

base_path = "D:/OneDrive/data/lab_data/Greyson_grasping/"
file_list = fnmatch.filter(os.listdir(base_path), "*.mat")
file_list = np.sort(file_list)
print("Here are the files under this folder: \n")
print(file_list)

Here are the files under this folder: 

['20190815_Greyson_Key_001.mat' '20190815_Greyson_Key_002.mat'
 '20190911_Greyson_Key_003.mat' '20190911_Greyson_Key_004.mat'
 '20190911_Greyson_PG_001.mat' '20190911_Greyson_PG_002.mat']


In [3]:
file_number = 0
bin_size = 0.05
dataset = lab_data(base_path, file_list[file_number])
dataset.update_bin_data(bin_size)

D:/OneDrive/data/lab_data/Greyson_grasping/20190815_Greyson_Key_001.mat
Monkey: Greyson
Task: multi_gadget
Collected on 2019/8/15 20:9:56.486 
Raw file name is 20190815_Greyson_Key_001
The array is in M1
There are 96 neural channels
Sorted? 0
There are 12 EMG channels
Current bin width is 0.0010 seconds
The name of each EMG channel:
EMG_FCR
EMG_FDS1
EMG_FDP3
EMG_PT
EMG_APB
EMG_FPB
EMG_LUM
EMG_1DI
EMG_EPL
EMG_SUP
EMG_ECU
EMG_EDC1
The dataset lasts 900.0090 seconds
There are 126 trials
In 113 trials the monkey got reward
The new bin width is 0.0500 s


### Dividing training and testing sets and formatting data for decoder training
Without removing samples between trials

In [4]:
from wiener_filter import format_data
n_lags = 8
train_test_ratio = 0.5 # It means 60% data in this set for training, and 40% for testing
total_sample = np.size(dataset.spike_counts, 0)
train_x = dataset.spike_counts[:int(train_test_ratio*total_sample), :] 
train_y = dataset.EMG[:int(train_test_ratio*total_sample), :]
test_x = dataset.spike_counts[int(train_test_ratio*total_sample): , :] 
test_y = dataset.EMG[int(train_test_ratio*total_sample): , :]
train_x, train_y = format_data(train_x, train_y, n_lags)
test_x, test_y = format_data(test_x, test_y, n_lags)
print("There are %d training samples. " % np.size(train_x, 0))
print("There are %d testing samples. " % np.size(test_x, 0))

There are 8992 training samples. 
There are 8992 testing samples. 


### Training a linear Wiener filter based decoder without L2 regularization

In [5]:
from wiener_filter import train_wiener_filter, test_wiener_filter, vaf
H_reg = train_wiener_filter(train_x, train_y, l2 = 0)
test_y_pred = test_wiener_filter(test_x, H_reg)
print("VAF for linear Wiener filter is %.3f" % vaf(test_y, test_y_pred))

VAF for linear Wiener filter is 0.760


### Training a linear Wiener filter based decoder with L2 regularization

In [6]:
H_reg = train_wiener_filter(train_x, train_y, l2 = 1)
test_y_pred = test_wiener_filter(test_x, H_reg)
print("VAF for linear Wiener filter is %.3f" % vaf(test_y, test_y_pred))

Sweeping ridge regularization using CV decoding on train data
Testing c= 10.0
Testing c= 16.237767391887218
Testing c= 26.366508987303583
Testing c= 42.81332398719393
Testing c= 69.51927961775606
Testing c= 112.88378916846884
Testing c= 183.29807108324357
Testing c= 297.63514416313194
Testing c= 483.2930238571752
Testing c= 784.7599703514607
Testing c= 1274.2749857031336
Testing c= 2069.13808111479
Testing c= 3359.818286283781
Testing c= 5455.594781168515
Testing c= 8858.667904100823
Testing c= 14384.498882876629
Testing c= 23357.21469090121
Testing c= 37926.90190732246
Testing c= 61584.82110660255
Testing c= 100000.0
VAF for linear Wiener filter is 0.797


### Training a nonlinear Wiener filter based decoder without L2 regularization
Sometimes nonlinear version performs better, but not necessarily

In [10]:
from wiener_filter import train_nonlinear_wiener_filter, test_nonlinear_wiener_filter
H_reg, res_lsq = train_nonlinear_wiener_filter(train_x, train_y, l2 = 0)
test_y_pred = test_nonlinear_wiener_filter(test_x, H_reg, res_lsq)
print("VAF for nonlinear Wiener filter is %.3f" % vaf(test_y, test_y_pred))

VAF for nonlinear Wiener filter is 0.768


### Training a nonlinear Wiener filter based decoder with L2 regularization

In [11]:
from wiener_filter import train_nonlinear_wiener_filter, test_nonlinear_wiener_filter
H_reg, res_lsq = train_nonlinear_wiener_filter(train_x, train_y, l2 = 1)
test_y_pred = test_nonlinear_wiener_filter(test_x, H_reg, res_lsq)
print("VAF for nonlinear Wiener filter is %.3f" % vaf(test_y, test_y_pred))

Sweeping ridge regularization using CV decoding on train data
Testing c= 10.0
Testing c= 16.237767391887218
Testing c= 26.366508987303583
Testing c= 42.81332398719393
Testing c= 69.51927961775606
Testing c= 112.88378916846884
Testing c= 183.29807108324357
Testing c= 297.63514416313194
Testing c= 483.2930238571752
Testing c= 784.7599703514607
Testing c= 1274.2749857031336
Testing c= 2069.13808111479
Testing c= 3359.818286283781
Testing c= 5455.594781168515
Testing c= 8858.667904100823
Testing c= 14384.498882876629
Testing c= 23357.21469090121
Testing c= 37926.90190732246
Testing c= 61584.82110660255
Testing c= 100000.0
VAF for nonlinear Wiener filter is 0.793
